In [1]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.chains.llm import LLMChain
from langchain.chains import ConversationChain

In [2]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv()

True

# 创建提示模板

In [3]:
physics_template = """You are a very smart physics professor.
You are great at answering questions about physics in a concise and easy to understand manner.
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""

math_template = """You are a very good mathematician. You are great at answering math questions.
You are so good because you are able to break down hard problems into their component parts,
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]

# 创建目标链

In [4]:
llm = ChatOpenAI(model="gpt-4o")

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

default_chain = ConversationChain(llm=llm, output_key="text")

/var/folders/rq/kthvngcn3r769fzgn106ymd00000gn/T/ipykernel_92723/68843885.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/var/folders/rq/kthvngcn3r769fzgn106ymd00000gn/T/ipykernel_92723/68843885.py:11: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  default_chain = ConversationChain(llm=llm, output_key="text")
/opt/anaconda3/envs/rag_book/lib/python3.11/site-packages/pydantic/main.py:214: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validat

# 创建路由链

In [5]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# 创建 MultiPromptChain

In [6]:
from langchain.chains.router import MultiPromptChain

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

/var/folders/rq/kthvngcn3r769fzgn106ymd00000gn/T/ipykernel_92723/883156345.py:3: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(


# 调用链并获取结果

In [7]:
result = chain.run("What is black body radiation?")
print(result)

/var/folders/rq/kthvngcn3r769fzgn106ymd00000gn/T/ipykernel_92723/898669921.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run("What is black body radiation?")




> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.
Black body radiation refers to the electromagnetic radiation emitted by an idealized object called a black body. A black body is an object that absorbs all incident radiation, regardless of frequency or angle, making it a perfect absorber. When this object is at a constant temperature, it also emits radiation with a characteristic spectrum that depends only on its temperature, not on its shape or material composition.

Key features of black body radiation include:
- It has a continuous spectrum that shifts towards shorter wavelengths as the temperature increases, which is described by Wien's Displacement Law.
- The total energy emitted per unit surface area is proportional to the fourth power of the black body’s temperature, as described by the Stefan-Boltzmann Law.
- The shape of the spectrum is described by Planck’s Law, which accounts for quantum mechanical effects and ex